参考資料
* [標準ライブラリ](https://docs.python.org/ja/3/library/sys.html)
* [PyMOTW-3](https://pymotw.com/3/sys/index.html)

> このモジュールでは、インタプリタで使用・管理している変数や、インタプリタの動作に深く関連する関数を定義しています。  
> このモジュールは常に利用可能です。

## インタプリタ設定

### バージョン情報

In [1]:
import sys

print('Version info:')
print()
print('sys.version      =', repr(sys.version))
print('sys.version_info =', sys.version_info)
print('sys.hexversion   =', hex(sys.hexversion)) # 単精度整数にエンコードされたバージョン番号
print('sys.api_version  =', sys.api_version) # C API バージョン

Version info:

sys.version      = '3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39) \n[Clang 6.0 (clang-600.0.57)]'
sys.version_info = sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0)
sys.hexversion   = 0x30806f0
sys.api_version  = 1013


```bash
Python 3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39) 
[Clang 6.0 (clang-600.0.57)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 
```
pythonを起動する時、表示するバーション情報は`sys.version`

sys.version_info
* major
* minor
* micro
* releaselevel
* serial

`major.minor.micro`はPythonのバーションを示す  
`releaselevel`は'alpha', 'beta', 'candidate', 'final'の一つ

In [2]:
import sys

print('This interpreter was built for:', sys.platform)

This interpreter was built for: darwin


|システム|platform の値|
|:-:|:-:|
|Linux|'linux'|
|MacOS|'darwin'|
|Windows|'win32'|

### インタプリタ実装

In [4]:
import sys


print('Name:', sys.implementation.name)
print('Version:', sys.implementation.version) # CPython以外の場合は"sys.version_info"と違う
print('Cache tag:', sys.implementation.cache_tag)

Name: cpython
Version: sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0)
Cache tag: cpython-38


### コマンドライン選択肢

In [6]:
!python3 -h

usage: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Options and arguments (and corresponding environment variables):
-b     : issue warnings about str(bytes_instance), str(bytearray_instance)
         and comparing bytes/bytearray with str. (-bb: issue errors)
-B     : don't write .pyc files on import; also PYTHONDONTWRITEBYTECODE=x
-c cmd : program passed in as string (terminates option list)
-d     : debug output from parser; also PYTHONDEBUG=x
-E     : ignore PYTHON* environment variables (such as PYTHONPATH)
-h     : print this help message and exit (also --help)
-i     : inspect interactively after running script; forces a prompt even
         if stdin does not appear to be a terminal; also PYTHONINSPECT=x
-I     : isolate Python from the user's environment (implies -E and -s)
-m mod : run library module as a script (terminates option list)
-O     : remove assert and __debug__-dependent statements; add .opt-1 bef

### Unicodeデフォルト

In [13]:
import sys

print('Default encoding     :', sys.getdefaultencoding())
print('File system encoding :', sys.getfilesystemencoding())

Default encoding     : utf-8
File system encoding : utf-8


### 対話的プロンプト

In [ ]:
```python
>>> import sys
>>> sys.ps1
'>>> '
>>> sys.ps2
'... '
```

> 任意の`__str__`を定義したオブジェクトはプロンプトに利用できる

In [ ]:
```python
>>> import sys
>>> class IPythonStyle:
...     def __init__(self):
...         self.count = 0
...     
...     def __str__(self):
...         self.count += 1
...         return f"In [{self.count}]:"
... 
>>> sys.ps1=IPythonStyle()
In [1]:
In [2]:
In [3]:
```

### Install Location

In [22]:
import sys

print('Interpreter executable:')
# プロンプトの場所、特に仮想環境を利用する際正しいプロンプトを利用しているかどうか確認できる
print(sys.executable)
print('\nInstallation prefix:')
# プロンプトがインストールされる場所
print(sys.prefix)

Interpreter executable:
/Library/Frameworks/Python.framework/Versions/3.8/bin/python3

Installation prefix:
/Library/Frameworks/Python.framework/Versions/3.8


## Runtime Environment

### コマンドラインパラメータ

In [25]:
%%writefile foo.py

import sys

print("Arguments: ", sys.argv)

Writing foo.py


In [29]:
%run foo.py

Arguments:  ['foo.py']


In [30]:
%run foo.py -v foo blabababa

Arguments:  ['foo.py', '-v', 'foo', 'blabababa']


In [31]:
!rm foo.py

### Input and Output Streams

In [33]:
%%writefile foo.py

import sys

print('STATUS: Reading from stdin', file=sys.stderr)
 # stdinは普通input,でもpipelineを経由で他のプログラムから取得できる
data = sys.stdin.read()

print('STATUS: Writing data to stdout', file=sys.stderr)

sys.stdout.write(data)　# pipelineへ渡ることができる
sys.stdout.flush()

print('STATUS: Done', file=sys.stderr)

Writing foo.py


In [35]:
!cat foo.py | python3 -u foo.py

STATUS: Reading from stdin
STATUS: Writing data to stdout

import sys

print('STATUS: Reading from stdin', file=sys.stderr)

data = sys.stdin.read()

print('STATUS: Writing data to stdout', file=sys.stderr)

sys.stdout.write(data)
sys.stdout.flush()

print('STATUS: Done', file=sys.stderr)
STATUS: Done


In [36]:
!rm foo.py

### Returning Status

In [38]:
%%writefile foo.py

import sys

exit_code = int(sys.argv[1])
sys.exit(exit_code)

Writing foo.py


In [39]:
!python3 foo.py 0; echo "Exited $?"

Exited 0


In [40]:
!python3 foo.py 1; echo "Exited $?"

Exited 1


In [41]:
!rm foo.py

## メモリ管理と限界値

### 参照カウント

In [44]:
import sys

one = []
print('At start         :', sys.getrefcount(one))

two = one

print('Second reference :', sys.getrefcount(one))

del two

print('After del        :', sys.getrefcount(one))

At start         : 2
Second reference : 3
After del        : 2


> 0になったら`garbage collector(gc)`によりメモリから削除される

### オブジェクト大きさ

In [46]:
import sys


class MyClass:
    pass


objects = [
    [], (), {}, 'c', 'string', b'bytes', 1, 2.3,
    MyClass, MyClass(),
]

for obj in objects:
    print(f'{type(obj).__name__:>10} : {sys.getsizeof(obj)}') # 単位: Bytes

      list : 56
     tuple : 40
      dict : 64
       str : 50
       str : 55
     bytes : 38
       int : 28
     float : 24
      type : 1064
   MyClass : 48


In [47]:
import sys


class WithoutAttributes:
    pass


class WithAttributes:
    def __init__(self):
        self.a = 'a'
        self.b = 'b'
        return


without_attrs = WithoutAttributes()
print('WithoutAttributes:', sys.getsizeof(without_attrs))

with_attrs = WithAttributes()
print('WithAttributes:', sys.getsizeof(with_attrs))

WithoutAttributes: 48
WithAttributes: 48


> 属性は計算されていないのため間違った結果となる

In [49]:
import sys


class WithAttributes:
    def __init__(self):
        self.a = 'a'
        self.b = 'b'
        return

    def __sizeof__(self):
        return object.__sizeof__(self) + \
            sum(sys.getsizeof(v) for v in self.__dict__.values())

my_inst = WithAttributes()
print(sys.getsizeof(my_inst))

148


### 再帰

In [1]:
import sys

print('Initial limit:', sys.getrecursionlimit())

sys.setrecursionlimit(56)

print('Modified limit:', sys.getrecursionlimit())


def generate_recursion_error(i):
    print(f'generate_recursion_error({i})')
    generate_recursion_error(i + 1)


try:
    generate_recursion_error(1)
except RuntimeError as err:
    print('Caught exception:', err)

Initial limit: 3000
Modified limit: 56
generate_recursion_error(1)
generate_recursion_error(2)
generate_recursion_error(3)
generate_recursion_error(4)
generate_recursion_error(5)
generate_recursion_error(6)
generate_recursion_error(7)
generate_recursion_error(8)
generate_recursion_error(9)
generate_recursion_error(10)
generate_recursion_error(11)
generate_recursion_error(12)
generate_recursion_error(13)
Caught exception: maximum recursion depth exceeded while calling a Python object


### 最大値

In [3]:
import sys

print('maxsize   :', sys.maxsize)
print('maxunicode:', sys.maxunicode)

maxsize   : 9223372036854775807
maxunicode: 1114111


### Floating Point Values

In [5]:
import sys

print('Smallest difference (epsilon):', sys.float_info.epsilon)
print()
print('Digits (dig)              :', sys.float_info.dig)
print('Mantissa digits (mant_dig):', sys.float_info.mant_dig)
print()
print('Maximum (max):', sys.float_info.max)
print('Minimum (min):', sys.float_info.min)
print()
print('Radix of exponents (radix):', sys.float_info.radix)
print()
print('Maximum exponent for radix (max_exp):',
      sys.float_info.max_exp)
print('Minimum exponent for radix (min_exp):',
      sys.float_info.min_exp)
print()
print('Max. exponent power of 10 (max_10_exp):',
      sys.float_info.max_10_exp)
print('Min. exponent power of 10 (min_10_exp):',
      sys.float_info.min_10_exp)
print()
print('Rounding for addition (rounds):', sys.float_info.rounds)

Smallest difference (epsilon): 2.220446049250313e-16

Digits (dig)              : 15
Mantissa digits (mant_dig): 53

Maximum (max): 1.7976931348623157e+308
Minimum (min): 2.2250738585072014e-308

Radix of exponents (radix): 2

Maximum exponent for radix (max_exp): 1024
Minimum exponent for radix (min_exp): -1021

Max. exponent power of 10 (max_10_exp): 308
Min. exponent power of 10 (min_10_exp): -307

Rounding for addition (rounds): 1


### 整数値

In [7]:
import sys

print('Number of bits used to hold each digit:',
      sys.int_info.bits_per_digit)
print('Size in bytes of C type used to hold each digit:',
      sys.int_info.sizeof_digit)

Number of bits used to hold each digit: 30
Size in bytes of C type used to hold each digit: 4


## Modules and Imports

### Imported Modules

In [8]:
import sys
import textwrap

names = sorted(sys.modules.keys())
name_text = ', '.join(names)

print(textwrap.fill(name_text, width=64))

IPython, IPython.core, IPython.core.alias,
IPython.core.application, IPython.core.async_helpers,
IPython.core.autocall, IPython.core.builtin_trap,
IPython.core.compilerop, IPython.core.completer,
IPython.core.completerlib, IPython.core.crashhandler,
IPython.core.debugger, IPython.core.display,
IPython.core.display_trap, IPython.core.displayhook,
IPython.core.displaypub, IPython.core.error,
IPython.core.events, IPython.core.excolors,
IPython.core.extensions, IPython.core.formatters,
IPython.core.getipython, IPython.core.history,
IPython.core.hooks, IPython.core.inputtransformer2,
IPython.core.interactiveshell, IPython.core.latex_symbols,
IPython.core.logger, IPython.core.macro, IPython.core.magic,
IPython.core.magic_arguments, IPython.core.magics,
IPython.core.magics.auto, IPython.core.magics.basic,
IPython.core.magics.code, IPython.core.magics.config,
IPython.core.magics.display, IPython.core.magics.execution,
IPython.core.magics.extension, IPython.core.magics.history,
IPython.core.mag

### Built-in Modules

In [11]:
import sys
import textwrap

name_text = ', '.join(sorted(sys.builtin_module_names))

print(textwrap.fill(name_text, width=64))

_abc, _ast, _codecs, _collections, _functools, _imp, _io,
_locale, _operator, _signal, _sre, _stat, _string, _symtable,
_thread, _tracemalloc, _warnings, _weakref, atexit, builtins,
errno, faulthandler, gc, itertools, marshal, posix, pwd, sys,
time, xxsubtype


### Import Path

In [13]:
import sys

for d in sys.path:
    print(d)

/Users/maozhongfu/GitHub/Python/Runtime Features
/Users/maozhongfu/.vscode/extensions/ms-python.python-2020.10.332292344/pythonFiles
/Users/maozhongfu/.vscode/extensions/ms-python.python-2020.10.332292344/pythonFiles/lib/python
/Library/Frameworks/Python.framework/Versions/3.8/lib/python38.zip
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload

/Users/maozhongfu/Library/Python/3.8/lib/python/site-packages
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/aeosa
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/extensions
/Users/maozhongfu/.ipython
